In [14]:
# !source ../setup.sh

In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import xgboost as xgb
from utils import DataWrapper, Visualizer
from src.prepare_dataset import prepare_data
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
# from src.models.baseline.tree_models import XGBoost, LightGBM
# from src.models.baseline.baseline_models import LinearModel, KNN, SVM, DecisionTree, RandomForest
sys.path.append(os.path.dirname(os.getcwd()))
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'loguru'

In [4]:
prepare_data("../data/final_dataset.csv", "./tmp")

../data/final_dataset.csv


In [22]:
train = pd.read_csv("./tmp/train.csv")
val = pd.read_csv("./tmp/valid.csv")
test = pd.read_csv("./tmp/test.csv")
keep_tmstamp = False
if keep_tmstamp:
    train['tmstamp'] = train['tmstamp'].astype('category')
    val['tmstamp'] = val['tmstamp'].astype('category')
    test['tmstamp'] = test['tmstamp'].astype('category')
else:
    train = train.drop(columns=['tmstamp'])
    val = val.drop(columns=['tmstamp'])
    test = test.drop(columns=['tmstamp'])

In [23]:
X_train, y_train = train.loc[:, train.columns != 'measured_t'], train['measured_t']
X_val, y_val = val.loc[:, val.columns != 'measured_t'], val['measured_t']
X_test, y_test = test.loc[:, test.columns != 'measured_t'], test['measured_t']

In [24]:
X_train

,classroom_type,school_day,school_hours,break_time,power_on,outdoor_temperature,trackerwm_avg,shadowwm_avg,sunwm_avg,shadowbandwm_avg,...,ws_ms_s_wvt,winddir_d1_wvt,winddir_sd1_wvt,bp_mb_avg,uva_avg,uvb_avg,Month,Day,Hour,Minute
0,3,1,0,0,1,18.20144,667.206050,51.265274,230.79771,69.409560,...,5.075,247.1,20.590,997.78815,0.020623,0.009368,2,8,7,34
1,3,1,0,0,1,19.38780,751.833400,59.186935,322.10257,61.510685,...,5.238,247.0,24.800,997.79270,0.020280,0.009368,2,8,7,57
2,3,1,1,0,0,20.14053,812.355300,64.872480,412.76123,65.492730,...,5.788,249.3,27.400,997.81710,0.020623,0.009368,2,8,8,20
3,3,1,1,0,0,20.51689,834.311200,66.790970,451.86554,67.126980,...,5.188,240.8,24.800,997.83890,0.020280,0.009368,2,8,8,30
4,3,1,1,0,0,21.08144,864.837300,69.757614,513.46985,69.447914,...,5.638,249.7,25.010,997.82200,0.020623,0.009368,2,8,8,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260453,2,0,0,0,1,7.40326,0.051560,0.000000,0.00000,0.000000,...,0.491,25.9,1.814,1009.80237,0.908038,0.078340,9,17,1,11
260454,5,0,0,0,1,7.40326,0.051560,0.000000,0.00000,0.000000,...,0.491,25.9,1.814,1009.80237,0.908038,0.078340,9,17,1,11
260455,2,0,0,0,1,7.40326,0.058005,0.000000,0.00000,0.000000,...,0.222,28.9,0.922,1009.87460,0.908038,0.078340,9,17,1,12
260456,5,0,0,0,1,7.40326,0.058005,0.000000,0.00000,0.000000,...,0.222,28.9,0.922,1009.87460,0.908038,0.078340,9,17,1,12


In [25]:
# Create regression matrices
cat = False
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=cat)
dval_reg = xgb.DMatrix(X_val, y_val, enable_categorical=cat)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=cat)

In [26]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
evals = [(dtrain_reg, "train"), (dval_reg, "validation")]

n = 10000
model_xgb = xgb.train(
    params=params,
    dtrain=dtrain_reg,
    num_boost_round=n,
    evals=evals,
    verbose_eval=10,
    early_stopping_rounds=50
)

[0]	train-rmse:2.75470	validation-rmse:3.14472
[10]	train-rmse:1.39930	validation-rmse:1.99069
[20]	train-rmse:1.29225	validation-rmse:1.96701
[30]	train-rmse:1.25590	validation-rmse:1.95680
[40]	train-rmse:1.22652	validation-rmse:1.96331
[50]	train-rmse:1.20451	validation-rmse:1.95427
[60]	train-rmse:1.18595	validation-rmse:1.95595
[70]	train-rmse:1.17405	validation-rmse:1.96134
[80]	train-rmse:1.16317	validation-rmse:1.96079
[90]	train-rmse:1.15217	validation-rmse:1.97442
[100]	train-rmse:1.14413	validation-rmse:1.99003
[102]	train-rmse:1.14159	validation-rmse:1.99598


In [27]:
pred_xgb = model_xgb.predict(dtest_reg)
rmse_xgb = mean_squared_error(y_test, pred_xgb, squared=False)
r2_xgb = r2_score(y_test, pred_xgb)

print(f"RMSE of the XGBoost model: {rmse_xgb:.5f}")
print(f"R2 of the XGBoost model: {r2_xgb:.5f}")

RMSE of the XGBoost model: 4.59145
R2 of the XGBoost model: -0.85077


In [30]:
# Fitting Random Forest Regression to the dataset
model_rfr = RandomForestRegressor(n_estimators=100, random_state=0, oob_score=True)

# Fit the regressor with x and y data
model_rfr.fit(X_train, y_train)

RandomForestRegressor(oob_score=True, random_state=0)

In [31]:
pred_rfr = model_rfr.predict(X_test)
rmse_rfr = mean_squared_error(y_test, pred_rfr, squared=False)
r2_rfr = r2_score(y_test, pred_rfr)

print(f"RMSE of the Random Forest Regression model: {rmse_rfr:.5f}")
print(f"R2 of the Random Forest Regression model: {r2_rfr:.5f}")

RMSE of the Random Forest Regression model: 4.56435
R2 of the Random Forest Regression model: -0.82899
